<a href="https://colab.research.google.com/github/VigneshwaraChinnadurai/DL_Works/blob/master/Auto_Encoder/Auto_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
# This is just to import from G-Drive

#STEP-1: Install Import Libraries
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#STEP-2: Autheticate E-Mail ID

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#STEP-3: Get File from Drive using file-ID

#2.1 Get the file
downloaded = drive.CreateFile({'id':'1lHnYashy9uWRqyPAFShZ5XGcYNGi09k9'})
# replace the id with id of file you want to access
downloaded.GetContentFile('u1.base') 

#2.1 Get the file
downloaded = drive.CreateFile({'id':'14VX5_lWUH0TjciBsgxcwS2U7R9dm-Gpa'})
# replace the id with id of file you want to access
downloaded.GetContentFile('u1.test') 

In [0]:
# Preparing the training set and the test set
training_set = pd.read_csv('u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [6]:
type(training_set)

numpy.ndarray

In [7]:
len(training_set)

79999

In [8]:
len(test_set)

19999

In [9]:
training_set.shape

(79999, 4)

In [0]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [11]:
print(f'Total no of Users={nb_users}')
print(f'Movie Count={nb_movies}')

Total no of Users=943
Movie Count=1682


In [0]:
# Converting the data into an array with users in lines and movies in columns (Like tokenization in NLP)
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [13]:
len(training_set)

943

In [14]:
len(training_set[0])

1682

In [0]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [16]:
training_set.type

<function Tensor.type>

In [0]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [0]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [19]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.7714)
epoch: 2 loss: tensor(1.0967)
epoch: 3 loss: tensor(1.0534)
epoch: 4 loss: tensor(1.0383)
epoch: 5 loss: tensor(1.0308)
epoch: 6 loss: tensor(1.0266)
epoch: 7 loss: tensor(1.0238)
epoch: 8 loss: tensor(1.0220)
epoch: 9 loss: tensor(1.0205)
epoch: 10 loss: tensor(1.0196)
epoch: 11 loss: tensor(1.0187)
epoch: 12 loss: tensor(1.0185)
epoch: 13 loss: tensor(1.0178)
epoch: 14 loss: tensor(1.0177)
epoch: 15 loss: tensor(1.0172)
epoch: 16 loss: tensor(1.0167)
epoch: 17 loss: tensor(1.0165)
epoch: 18 loss: tensor(1.0166)
epoch: 19 loss: tensor(1.0164)
epoch: 20 loss: tensor(1.0161)
epoch: 21 loss: tensor(1.0162)
epoch: 22 loss: tensor(1.0159)
epoch: 23 loss: tensor(1.0157)
epoch: 24 loss: tensor(1.0157)
epoch: 25 loss: tensor(1.0156)
epoch: 26 loss: tensor(1.0157)
epoch: 27 loss: tensor(1.0152)
epoch: 28 loss: tensor(1.0151)
epoch: 29 loss: tensor(1.0140)
epoch: 30 loss: tensor(1.0118)
epoch: 31 loss: tensor(1.0107)
epoch: 32 loss: tensor(1.0075)
epoch: 33 loss: t

In [22]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.9497704769461011
